In this notebook, I will use the information about IA to refine F3. Initially, I buildt F3 using FEF25-75%FEV1 against FEV1-based-AR. The latter variable contains only the aspect of AR as measured by FEV1. In CF, high AR usually correlates with high IA. We can use this correlation to refine the uncertainty present in the FEV1-based-AR. The corrected FEV1-based-AR can therefore be closer to the true AR. We can use it to improve the model fo F3.

In [ ]:
import src.data.breathe_data as br
import src.data.helpers as dh
import src.models.helpers as mh
import src.inference.helpers as ih
import src.modelling_fef2575.cpt_and_plots as cpt_and_plots
import src.models.var_builders as var_builders
import src.models.graph_builders as graph_builders
from src.inference.inf_algs import apply_bayes_net_bp, apply_factor_graph_bp
import src.models.cpts.helpers as cpth

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
(
    HFEV1,
    ecFEV1,
    AR,
    HO2Sat,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
    ecFEF2575prctecFEV1,
) = var_builders.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(
    180, 10, "Male"
)
# (
#     HFEV1,
#     uecFEV1,
#     ecFEV1,
#     AR,
#     HO2Sat,
#     O2SatFFA,
#     IA,
#     UO2Sat,
#     O2Sat,
#     ecFEF2575prctecFEV1
# ) = var_builders.o2sat_fev1_fef2575_point_in_time_model_noise_shared_healthy_vars_light(
#     180, 10, "Male"
# )

# Plan to refine F3

In [ ]:
# Need dataset with O2sat, FEV1, FEF25-75. Use as many datapoints as possible
# Infer FEV1-based-AR using FEV1
# Model F3 using FEV1-based-AR
# Infer FEV1-FEF2575-based-AR using FEV1 and FEF25-75
# Model F3 using FEV1-FEF2575-based-AR
# Infer IA using FEV1-FEF2575-based-AR
# Model AR-IA
# Infer IA-FEV1-FEF2575-based-AR using FEV1, FEF25-75, IA
# Model F3 using this new AR
# Compare the two models: compare the mean, median, std-percentiles plots of both -> std should be smaller

# Optionally repeat until std doesn't change

# Load data

In [ ]:
df = br.load_meas_from_excel(
    "BR_O2_FEV1_FEF2575_with_idx"
)

In [ ]:
# df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan")
df = df.drop(columns=["PEF", "ecPEF (L/s)", "PEF (L/s)"])
df = df.dropna(subset=["FEV1", "O2 Saturation", "FEF2575"])
# df["ecFEF2575%ecFEV1"] = df["FEF2575"] / df["FEV1"] * 100
df.head()

# Inference

In [ ]:
def infer_vars_and_get_back_df(
    df,
    variables_to_infer,
    observed_variables,
    ecFEF2575prctecFEV1_cpt=None,
    IA_cpt=None,
):
    """
    Infer AR, IA, HFEV1, HO2Sat fo each entry in the dataset, for the given observed variables as evidence
    """

    def infer_vars_for_ID(df):
        df.reset_index(inplace=True)

        # (
        #     HFEV1,
        #     ecFEV1,
        #     AR,
        #     HO2Sat,
        #     O2SatFFA,
        #     IA,
        #     UO2Sat,
        #     O2Sat,
        #     ecFEF2575prctecFEV1,
        # ) = var_builders.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(
        #     df.Height[0], df.Age[0], df.Sex[0]
        # )

        # If with noise model
        (
            HFEV1,
            uecFEV1,
            ecFEV1,
            AR,
            HO2Sat,
            O2SatFFA,
            IA,
            UO2Sat,
            O2Sat,
            ecFEF2575prctecFEV1,
        ) = var_builders.o2sat_fev1_fef2575_point_in_time_model_noise_shared_healthy_vars_light(
            df.Height[0], df.Age[0], df.Sex[0]
        )

        # Update cpt to custom one if provided
        if ecFEF2575prctecFEV1_cpt is not None:
            ecFEF2575prctecFEV1.set_cpt(ecFEF2575prctecFEV1_cpt)
        if IA_cpt is None:
            # model = graph_builders.fev1_fef2575_o2sat_point_in_time_factor_graph(
            #     HFEV1,
            #     ecFEV1,
            #     AR,
            #     HO2Sat,
            #     O2SatFFA,
            #     IA,
            #     UO2Sat,
            #     O2Sat,
            #     ecFEF2575prctecFEV1,
            #     False,
            # )
            model = graph_builders.fev1_fef2575_o2sat_point_in_time_noise_factor_graph(
                HFEV1,
                uecFEV1,
                ecFEV1,
                AR,
                HO2Sat,
                O2SatFFA,
                IA,
                UO2Sat,
                O2Sat,
                ecFEF2575prctecFEV1,
                False,
            )
            inf_alg = apply_factor_graph_bp(model)
        else:
            # Else we model AR causing IA with the given CPT
            IA.set_cpt(IA_cpt)
            # Since this introduces a loop we have to use a Bayes Net to run the inference
            # model = graph_builders.fev1_o2sat_fef2575_point_in_time_model(
            #     HFEV1,
            #     ecFEV1,
            #     AR,
            #     HO2Sat,
            #     O2SatFFA,
            #     IA,
            #     UO2Sat,
            #     O2Sat,
            #     ecFEF2575prctecFEV1,
            #     False,
            # )
            model = graph_builders.fev1_fef2575_o2sat_point_in_time_noise_factor_graph(
                HFEV1,
                uecFEV1,
                ecFEV1,
                AR,
                HO2Sat,
                O2SatFFA,
                IA,
                UO2Sat,
                O2Sat,
                ecFEF2575prctecFEV1,
                False,
            )
            inf_alg = apply_bayes_net_bp(model)

        def infer_and_unpack(row):
            # Build evidence
            evidence = [
                [obs_var, row[obs_var.get_colname()]] for obs_var in observed_variables
            ]

            if IA_cpt is None:
                res = ih.infer_on_factor_graph(
                    inf_alg,
                    variables_to_infer,
                    evidence,
                )
            else:
                # Infer on Bayes net
                res = ih.infer(
                    inf_alg,
                    variables_to_infer,
                    evidence,
                )

            res_values = (res[var.name].values for var in variables_to_infer)

            return row["Date Recorded"], *res_values

        res = df.apply(infer_and_unpack, axis=1)
        return res

    variables_to_infer_dict = {
        i + 1: variables_to_infer[i].get_abbr() for i in range(len(variables_to_infer))
    }
    variables_to_infer_dict[0] = "Date Recorded"

    resraw = df.groupby("ID").apply(infer_vars_for_ID)
    # resraw = df.iloc[np.r_[10:13, 3000:3007]].groupby("ID").apply(infer_vars_for_ID)
    res = (
        resraw.apply(pd.Series)
        .reset_index()
        .rename(columns=variables_to_infer_dict)
        .drop(columns="level_1")
    )

    for var in variables_to_infer:
        res[f"{var.get_abbr()} mean"] = res[var.get_abbr()].apply(
            lambda x: var.get_mean(x)
        )

    return res

## Model F3 using AR inferred from FEV1

In [ ]:
# Infer FEV1-based-AR using FEV1

# inf_res_df = infer_vars_and_get_back_df(
#     df,
#     # df.iloc[np.r_[10:1300, 3000:4007, 10000:11000]],
#     variables_to_infer=[AR, IA],
#     observed_variables=[ecFEV1, O2Sat],
#     ecFEF2575prctecFEV1_cpt=None,
# )
# # Merge the inferred AR back to the original dataframe
# df1 = pd.merge(df, inf_res_df, on=["ID", "Date Recorded"], how="inner")
# df1.to_excel(
#     dh.get_path_to_main()
#     + "ExcelFiles/BR/Refining_F3/infer_FEV1-based-AR_using_O2Sat_FEV1_noise_light.xlsx",
#     index=False,
# )

df1 = dh.load_excel(
    dh.get_path_to_main()
    + "ExcelFiles/BR/Refining_F3/infer_AR_using_O2Sat_ecFEV1.xlsx",
    ["AR"],
)

In [ ]:
# Model F3 using FEV1-based-AR
cpt_f3, df_f3, _ = cpt_and_plots.model_f3(
    df1,
    AR,
    "Airway resistance midbin in % (obs. var.: ecFEV1, O2 saturation)",
    save=True,
)

## Model F3 using AR inferred from FEV1 and FEF25-75 (using F3 to model F3)

In [ ]:
# Infer FEV1-FEF2575-based-AR
# inf_res_df_2 = infer_vars_and_get_back_df(
#     df,
#     # df.iloc[np.r_[10:1300, 3000:4007, 10000:11000]],
#     variables_to_infer=[AR, IA],
#     observed_variables=[ecFEV1, O2Sat, ecFEF2575prctecFEV1],
#     ecFEF2575prctecFEV1_cpt=cpt_f3,
# )
# # # Merge the inferred AR back to the original dataframe
# df2 = pd.merge(df, inf_res_df_2, on=["ID", "Date Recorded"], how="inner")
# df2.to_excel(
#     dh.get_path_to_main()
#     + "ExcelFiles/BR/Refining_F3/infer_FEV1-FEF2575-based-AR_using_O2Sat_FEV1_FEF2575.xlsx",
#     index=False,
# )

df2 = dh.load_excel(
    dh.get_path_to_main()
    + "ExcelFiles/BR/Refining_F3/infer_FEV1-FEF2575-based-AR_using_O2Sat_FEV1_FEF2575.xlsx",
    ["AR"],
)

In [ ]:
# Model F3 using FEV1-FEF2575-based-AR

## TODO: why it gives nan when it's the first time I run it?
cpt_f3_2, df_f3_2, _ = cpt_and_plots.model_f3(df2, AR, "FEV1-FEF2575-based-AR")

## Model AR-IA using AR inferred with FEV1 and FEF25-75 (using F3 refined with F3 once)

In [ ]:
# Infer IA, AR using FEV1-FEF2575-based-AR

# inf_res_df_3 = infer_vars_and_get_back_df(
#     df,
#     # df.iloc[np.r_[10:1300, 3000:4007, 10000:11000]],
#     variables_to_infer=[IA, AR],
#     observed_variables=[ecFEV1, O2Sat, ecFEF2575prctecFEV1],
#     ecFEF2575prctecFEV1_cpt=cpt_f3_2,
# )
# df3 = pd.merge(df, inf_res_df_3, on=["ID", "Date Recorded"], how="inner")
# df3.to_excel(
#     dh.get_path_to_main()
#     + "ExcelFiles/BR/Refining_F3/infer_FEV1-FEF2575-based-AR-and-IA_using_O2Sat_FEV1_FEF2575.xlsx",
#     index=False,
# )

df3 = dh.load_excel(
    dh.get_path_to_main()
    + "ExcelFiles/BR/Refining_F3/infer_FEV1-FEF2575-based-AR-and-IA_using_O2Sat_FEV1_FEF2575.xlsx",
    ["AR", "IA"],
)

In [ ]:
# Model the relationship between AR and IA, and add it to the model

AR_bin_width = 4
n_samples = 100

df_sampled_3 = cpt_and_plots.get_sampled_df_and_statistics_df_for_IA(
    df3, n_samples, AR, AR_bin_width, IA
)

# cpt_and_plots.plot_F3_mean_and_percentiles_per_AR_bin(df_f3, ar_col, y_col, save=True)

ar_col = "FEV1-FEF2575-based-AR"
cpt_ia_ar = cpt_and_plots.calc_plot_cpt_IA_given_AR(
    df_sampled_3, AR, AR_bin_width, ar_col, IA, n_samples, save=True, debug=True
)

# The four last histograms have too few data to be reliable
# Replacing them by the -11th
idx_sixty_six_from_back = AR.card - AR.get_bin_for_value(66)[1]
print(AR.get_bins_str()[-idx_sixty_six_from_back])
for j in range(idx_sixty_six_from_back - 1):
    cpt_ia_ar[:, -j - 1] = cpt_ia_ar[:, -idx_sixty_six_from_back]

# Renormalise the CPT
cpt_ia_ar = cpt_ia_ar / cpt_ia_ar.sum(axis=0)

cpt_and_plots.check_IA_cpt(cpt_ia_ar, IA, AR, True)

In [ ]:
import src.models.cpts.helpers as cpt


def get_cpt(vars, suffix=None):
    path_to_folder = dh.get_path_to_src() + "models/cpts/"
    var_spec = map(
        lambda var: f"{cpt.name_to_abbr(var.name)}_{var.a}_{var.b}_{var.bin_width}",
        vars,
    )
    filename = "_".join(var_spec)

    if suffix is not None:
        filename = filename + suffix

    arr = np.load(f"{path_to_folder}{filename}.npy")

    assert len(vars) == len(arr.shape)
    return arr

## Model F3 using AR inferred from AR-IA factor and FEV1

In [ ]:
# # Infer FEV1-based-AR with IA factor using FEV1

# # Longer inference because of bayes net
# inf_res_df_4 = infer_vars_and_get_back_df(
#     df,
#     variables_to_infer=[AR],
#     observed_variables=[ecFEV1, O2Sat],
#     ecFEF2575prctecFEV1_cpt=None,
#     IA_cpt=get_cpt([IA, AR], "_Dirichlet_prior"),
# )
# df4 = pd.merge(df, inf_res_df_4, on=["ID", "Date Recorded"], how="inner")
# # F4 is the IA-AR factor
# df4.to_excel(
#     dh.get_path_to_main()
#     + "ExcelFiles/BR/Refining_F3/infer_FEV1-based-AR_using_O2Sat_FEV1_F4-with-dirichlet-prior.xlsx",
#     index=False,
# )

df4 = dh.load_excel(
    dh.get_path_to_main()
    + "ExcelFiles/BR/infer_FEV1-based-AR_using_O2Sat_FEV1_F4-with-dirichlet-prior.xlsx",
    ["AR"],
)

In [ ]:
# Model F3 using FEV1-based-AR with IA factor
cpt_f3_4, df_f3_4, _ = cpt_and_plots.model_f3(df4, AR, "FEV1-based-AR with F4")

## Model F3 using AR inferred from FEV1, FEF25-75 (refined with itself once), and the AR-IA factor

In [ ]:
# Infer FEV1-FEF2575-based-AR with IA factor using FEV1 and F4

inf_res_df_5 = infer_vars_and_get_back_df(
    df,
    # df.iloc[np.r_[10:1300, 3000:4007]],
    variables_to_infer=[AR, IA],
    observed_variables=[ecFEV1, O2Sat, ecFEF2575prctecFEV1],
    ecFEF2575prctecFEV1_cpt=cpt_f3_2,
    IA_cpt=cpt_ia_ar,
)
df5 = pd.merge(df, inf_res_df_5, on=["ID", "Date Recorded"], how="inner")
df5.to_excel(
    dh.get_path_to_main()
    + "ExcelFiles/BR/Refining_F3/infer_FEV1-FEF2575-based-AR_using_O2Sat_FEV1_F4-with-dirichlet-prior.xlsx",
    index=False,
)

In [ ]:
df5 = pd.merge(df, inf_res_df_5, on=["ID", "Date Recorded"], how="inner")
df5.to_excel(
    dh.get_path_to_main()
    + "ExcelFiles/BR/Refining_F3/infer_FEV1-FEF2575-based-AR_using_O2Sat_FEV1_F4.xlsx",
    index=False,
)

In [ ]:
# Model F3 using FEV1-based-AR with IA factor
cpt_f3_5, df_f3_5, _ = cpt_and_plots.model_f3(df5, AR, "FEV1-FEF2575-based-AR with F4")

## Model F3 using AR from the two days model (FEV1, O2Sat)

In [ ]:
df_meas = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_with_idx")

In [ ]:
df6_tmp = dh.load_excel(
    f"{dh.get_path_to_main()}/ExcelFiles/BR/Refining_F3/infer_AR_with_two_days_model_O2Sat_FEV1.xlsx",
    [AR.name],
    ["Day"],
).drop(columns=["Unnamed: 0", HO2Sat.name, IA.name, HFEV1.name])

In [ ]:
df6 = df6_tmp.merge(
    df_meas, left_on=["ID", "Day"], right_on=["ID", "Date Recorded"], how="inner"
)
print(df_meas.shape, "length raw meas")
print(df6_tmp.shape, "length two days model output")
print(df6.shape, "length merged")
df6 = df6.drop(
    columns=[
        "Day",
        "FEV1",
        "O2 Saturation",
        "FEF2575",
        # "ecFEV1",
        # "ecFEF2575",
        "Sex",
        "Height",
        "Age",
        "Predicted FEV1",
        "Healthy O2 Saturation",
        "ecFEV1 % Predicted",
        "FEV1 % Predicted",
        "O2 Saturation % Healthy",
        "idx ecFEV1 (L)",
        "idx ecFEF25-75 % ecFEV1 (%)",
        "idx O2 saturation (%)",
    ]
).rename(columns={AR.name: "AR"})

In [ ]:
df6.columns

In [ ]:
# Model F3
cpt_f3_6, df_f3_6, _ = cpt_and_plots.model_f3(
    df6,
    AR,
    "Airway resistance midbin in %<br>Inferred using the two days model (Obs. var.: ecFEV1, O2 saturation)",
)

## Model F3 using AR from two days model (ecFEV1), with add mult ecfev1 noise

In [ ]:
df7_tmp = dh.load_excel(
    f"{dh.get_path_to_main()}/ExcelFiles/BR/Refining_F3/infer_AR_using_two_days_model_ecFEV1_ecfev1noiseaddmult.xlsx",
    [AR.name],
    ["Day"],
).drop(columns=[HO2Sat.name, HFEV1.name])

In [ ]:
# df7_tmp.to_excel(
#     f"{dh.get_path_to_main()}/ExcelFiles/BR/Refining_F3/infer_AR_using_two_days_model_ecFEV1_ecfev1noiseaddmult.xlsx",
#     index=False,
# )

In [ ]:
df_meas = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

In [ ]:
df7 = df7_tmp.merge(
    df_meas, left_on=["ID", "Day"], right_on=["ID", "Date Recorded"], how="inner"
)
print(df_meas.shape, "length raw meas")
print(df7_tmp.shape, "length two days model output")
print(df7.shape, "length merged")
df7 = df7.drop(
    columns=[
        "Day",
        "FEV1",
        "O2 Saturation",
        "FEF2575",
        # "ecFEV1",
        # "ecFEF2575",
        "Sex",
        "Height",
        "Age",
        "Predicted FEV1",
        "Healthy O2 Saturation",
        "ecFEV1 % Predicted",
        "FEV1 % Predicted",
        "O2 Saturation % Healthy",
        "idx ecFEV1 (L)",
        "idx ecFEF25-75 % ecFEV1 (%)",
        "idx O2 saturation (%)",
    ]
).rename(columns={AR.name: "AR"})

In [ ]:
# Model F3
cpt_f3_7, df_f3_7, _ = cpt_and_plots.model_f3(
    df7,
    AR,
    "Airway resistance midbin in %<br>Two days model (ecFEV1, addmult ecfev1 noise)",
)

## Plot results

In [ ]:
def add_traces_F3_mean_and_percentiles_per_AR_bin(fig, df_f3, dash_style="solid"):
    fig.add_traces(
        go.Scatter(
            x=df_f3["AR midbin"],
            y=df_f3["mean"],
            mode="lines+markers",
            line=dict(color="blue", dash=dash_style),
            name=f"Mean",
        )
    )
    fig.add_traces(
        go.Scatter(
            x=df_f3["AR midbin"],
            y=df_f3["median"],
            mode="lines+markers",
            line=dict(color="purple", dash=dash_style),
            name=f"Median",
        )
    )
    fig.add_traces(
        go.Scatter(
            x=df_f3["AR midbin"],
            y=df_f3["p16"],
            mode="lines+markers",
            line=dict(color="red", dash=dash_style),
            name=f"16th prctile",
        )
    )
    fig.add_traces(
        go.Scatter(
            x=df_f3["AR midbin"],
            y=df_f3["p84"],
            mode="lines+markers",
            line=dict(color="red", dash=dash_style),
            name=f"84th prctile",
        )
    )
    fig.add_traces(
        go.Scatter(
            x=df_f3["AR midbin"],
            y=df_f3["p3"],
            mode="lines+markers",
            line=dict(color="green", dash=dash_style),
            name=f"3th prctile",
        )
    )
    fig.add_traces(
        go.Scatter(
            x=df_f3["AR midbin"],
            y=df_f3["p97"],
            mode="lines+markers",
            line=dict(color="green", dash=dash_style),
            name=f"97th prctile",
        )
    )
    return -1

In [ ]:
# Conclusion
# df_f3 - F3 from FEV1-based-AR
# df_f3_2 - F3 from FEV1-FEF2575-based-AR
# df_f3_4 - F3 from FEV1-based-AR with IA-AR factor
# F3 from FEV1-FEF2575-based-AR

# ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']

fig = go.Figure()
add_traces_F3_mean_and_percentiles_per_AR_bin(fig, df_f3, "dot")
dot_name = "AR from FEV1"
add_traces_F3_mean_and_percentiles_per_AR_bin(fig, df_f3_2, "dash")
dash_name = "AR from FEV1 and FEF25-75"
add_traces_F3_mean_and_percentiles_per_AR_bin(fig, df_f3_6, "solid")
solid_name = "AR from two days model (FEV1 and O2sat)"
fig.add_annotation(
    x=1.1,
    y=1.15,
    text=f"dot: {dot_name}<br>dash: {dash_name}<br>solid: {solid_name}",
    xref="paper",
    yref="paper",
    showarrow=False,
    font=dict(size=12),
)
fig.update_xaxes(
    title=f"Airway resistance midbin (%)",
    tickvals=np.floor(list(df_f3["AR midbin"].values)),
)
fig.update_traces(marker=dict(size=2))
fig.update_yaxes(title="ecFEF2575prctecFEV1 (%)")
title = f"F3 statistics"
fig.update_layout(title=title, width=1200, height=700)
# fig.write_image(
#     f"{dh.get_path_to_main()}PlotsBreathe/AR_modelling/CPT - Refined F3 - with Dirichlet prior.pdf"
# )

# Sanitize and save CPT P(ecFEF25-75|AR)

In [ ]:
cpt_f3_7.shape

In [ ]:
# From graph, the dat is safe until 71% AR included (after too few data, and curve go up)
idx_cut = AR.get_bin_for_value(73)[1]
cptsave = cpt_f3_7[:, :idx_cut]
# Replicate the last column to 100%
cptsave = np.hstack(
    (
        cptsave,
        np.tile(
            cptsave[:, -1].reshape(-1, 1), AR.get_bin_for_value(100)[1] - idx_cut + 1
        ),
    )
)

In [ ]:
cptsave.shape

In [ ]:
fig, title = cpth.plot_2d_cpt(cptsave, ecFEF2575prctecFEV1, AR, p_range=[0, 0.06], invert=True, height=4000, vspace=0.001, y_label_two_lines=True)
# Save in CPTs
fig.write_image(
    f"{dh.get_path_to_main()}PlotsBreathe/CPTs/CPT - P(AR|ecFEF25-75%ecFEV1) two days model (ecFEV1, addmult ecfev1 noise).pdf"
)
# fig.write_image(
#     f"{dh.get_path_to_main()}PlotsBreathe/CPTs/CPT - P(ecFEF25-75%ecFEV1|AR) two days model (ecFEV1, addmult ecfev1 noise).pdf"
# )
# fig.show()

In [ ]:
cpth.save_cpt(
    [ecFEF2575prctecFEV1, AR], cptsave, suffix="_ecfev1_2_days_model_add_mult_noise"
)

# Plot ecFEF25-75%ecFEV1 vs AR sample

In [ ]:
df_sampled, df_f3 = cpt_and_plots.get_sampled_df_and_statistics_df(df3, 1, AR)
df_sampled_tmp = df_sampled[df_sampled["ecFEF2575%ecFEV1"] < 200]
cpt_and_plots.plot_FEF2575_ratio_with_IA(
    df_sampled_tmp, "AR sample", "ecFEF2575%ecFEV1"
)

# Model the relationship between AR and IA

In [ ]:
df3 = dh.load_excel(
    dh.get_path_to_main()
    + "ExcelFiles/BR/Refining_F3/infer_FEV1-FEF2575-based-AR-and-IA_using_O2Sat_FEV1_FEF2575.xlsx",
    ["AR", "IA"],
)

## Relationship between IA and AR

In [ ]:
fig = px.scatter(df3, x="AR mean", y="IA mean", color="ID")
fig.update_traces(marker=dict(size=4))
fig.update_layout(width=1000, height=600, title=f"IA vs AR")
fig.show()

In [ ]:
# Why is there fewer points with high IA where AR is in 25-50%?
# Outlying IA values are specific to a few (5) individuals. We just don't have an individual which is in the 25-50%.
# It makes sense to add Dirichlet data prior
df3[(df3["AR mean"] < 30) & (df3["IA mean"] > 8)].drop(columns=["AR", "IA"])

## Relationship between IA and AR after sampling (+ Dirichlet input)

In [ ]:
AR_bin_width = 10  # Not necessary for sampling, hence putting a high value
n_samples = 20
df_sampled_tmp = cpt_and_plots.get_sampled_df_and_statistics_df_for_IA(
    df3, n_samples, AR, AR_bin_width, IA
)

In [ ]:
def add_generated_dirichlet_fake_data_prior(df, AR, IA, n):
    dirichlet_data = pd.DataFrame()
    for ar_low, ar_high in AR.get_bins_arr():
        for ia_low, ia_high in IA.get_bins_arr():
            dirichlet_sample = pd.DataFrame(
                {
                    "AR sample": np.random.uniform(ar_low, ar_high, n),
                    "IA sample": np.random.uniform(ia_low, ia_high, n),
                }
            )
            dirichlet_data = pd.concat([dirichlet_data, dirichlet_sample])

    df["Dirichlet"] = False
    dirichlet_data["Dirichlet"] = True

    return pd.concat([df, dirichlet_data])

In [ ]:
# Add dirichlet input
AR_bin_width = 4
IA_bin_width = 1
df_sampled_tmp = df_sampled_tmp[["IA sample", "AR sample"]]
df_sampled_tmp = cpt_and_plots.add_binned_up_var(
    df_sampled_tmp, "AR sample", "AR", AR_bin_width
)
df_sampled_tmp = cpt_and_plots.add_binned_up_var(
    df_sampled_tmp, "IA sample", "IA", IA_bin_width
)

# For each AR bin and for each IA bin, add a Dirichlet prior of 10 uniformly distributed datapoints
n_dirichlet = 1  # 20 samples with bin width of 10 = 4 samples with bin width of 2

# Using bins with full resolution
df_sampled_tmp_dirichet = add_generated_dirichlet_fake_data_prior(
    df_sampled_tmp, AR, IA, n_dirichlet
)
print(df_sampled_tmp.shape)

In [ ]:
fig = px.scatter(df_sampled_tmp, x="AR sample", y="IA sample", color="ID")
# fig = px.scatter(df_sampled_tmp_dirichet, x="AR sample", y="IA sample", color='Dirichlet')
fig.update_traces(marker=dict(size=2))
fig.update_layout(width=1000, height=600, title=f"IA vs AR, {n_samples} samples")
fig.show()

## Sample

In [ ]:
AR_bin_width = 4
IA_bin_width = 1
n_samples = 100
ar_col = "FEV1-FEF2575-based AR"

df_sampled_3 = cpt_and_plots.get_sampled_df_for_AR_IA(df3, n_samples, AR, IA)

## Add dirichlet prior

In [ ]:
# Add Dirichlet fake data
print(df_sampled_3.shape)
# 4m datapoints is way to much, we need to randomly pick a subset of 100k of it

n_dirichlet = 1
df_sampled_3_dirichlet = add_generated_dirichlet_fake_data_prior(
    df_sampled_3, AR, IA, n_dirichlet
)

n_samples_filtered = 300000
df_sampled_3_dirichlet_reduced = df_sampled_3_dirichlet.sample(n_samples_filtered)

fig = px.scatter(
    df_sampled_3_dirichlet_reduced, x="AR sample", y="IA sample", color="Dirichlet"
)
fig.update_traces(marker=dict(size=3))
fig.update_layout(
    width=1000,
    height=600,
    title=f"IA vs AR, {n_samples_filtered} samples drawn after a {n_samples}x supersampling",
)
fig.show()

In [ ]:
# Bin up variables
df_sample_fit = df_sampled_3_dirichlet
# df_sample_fit = df_sampled_3

df_sample_fit = cpt_and_plots.add_binned_up_var(
    df_sample_fit, "AR sample", "AR", AR_bin_width
)
df_sample_fit = cpt_and_plots.add_binned_up_var(
    df_sample_fit, "IA sample", "IA", IA_bin_width
)

ar_groups = np.sort(list(df_sample_fit["AR midbin"].unique()))
ar_groups

## Fit Boltzmann MLE/MSE, custom exp MSE, Log series MLE

In [ ]:
# MLE
import scipy.stats as stats
from scipy.optimize import minimize

cpt_ia_ar = np.zeros([IA.card, len(ar_groups), 4])

# ar_groups_with_n = df_sample_fit['AR midbin'].value_counts().sort_index()
# ar_group_index = list(map(lambda ar_value: f"AR={ar_value:g}%", ar_groups_with_n.index))
# ar_group_count = list(map(lambda ar_count: f"n={ar_count}", ar_groups_with_n.values))
# ar_group_legend = [f"AR midbin={ar_group_index:g}%<br>n={ar_group_count}" for ar_group_index, ar_group_count in zip(ar_groups_with_n.index, ar_groups_with_n.values)]


# f"AR={ar_group}%<br>n={df_tmp['IA sample'].shape[0]}

for fit_id in [1, 2, 3, 4]:

    fig = make_subplots(
        cols=2,
        rows=len(ar_groups),
        shared_xaxes="all",
        # y_title="Probability",
        x_title="Inactive alveoli midbin value (%)",
        # row_titles=ar_group_legend,
        column_titles=[
            "Probabilities given in linear scale",
            "Probabilities given in log scale",
        ],
        # vertical_spacing=0.02,
        horizontal_spacing=0.04,
    )

    for idx, ar_group in enumerate(ar_groups):
        df_tmp = df_sample_fit[df_sample_fit["AR midbin"] == ar_group].copy()

        # Create histogram data for IA, binned by IA bins
        s_ia_hist = df_tmp["IA midbin"].value_counts()

        # Add 10% of the data distributed evenly on each bin
        # print("dirichlet factor", round(s_ia_hist.sum() * 0.1 / IA.card))
        # s_ia_hist_dirichlet = s_ia_hist  # + round(s_ia_hist.sum() * 0.1 / IA.card)
        # s_ia_hist_dirichlet = s_ia_hist  + 100
        # s_ia_hist_norm = s_ia_hist_dirichlet / s_ia_hist_dirichlet.sum()
        s_ia_hist_norm = s_ia_hist / s_ia_hist.sum()

        x = np.array(s_ia_hist_norm.index)
        p = s_ia_hist_norm.values

        # The boltzmann distribution is defined for integer values (not float)
        # That means we can't use the midbin value of the IA bins (since it's usually a float)
        # Instead we'll designate the bin by its lower boundary
        # Input data: the midbin value (uniform approximation within each bin).
        # Output data fetched using the bin's lower boundary value

        def get_y_for_fit(fit_id):
            # Boltzmann fit with MLE
            if fit_id == 1:
                bounds = [(0, 2), (IA.card, IA.card)]
                result = stats.fit(
                    stats.boltzmann, df_tmp["IA midbin"], bounds, method="mle"
                )
                dist_name = "Boltzmann"
                params_str = f"param={result.params.lambda_:.4g}"
                print(f"{result}")
                y_fit = stats.boltzmann(*result.params).pmf(IA.bins)

            # Log series fit with MLE
            if fit_id == 2:
                bounds = [(0, 1)]
                result = stats.fit(
                    stats.logser, df_tmp["IA midbin"].to_numpy() + 1, bounds
                )
                dist_name = "Log-Series"
                params_str = f"param={result.params.p:.4g}"
                print(f"{result}")
                y_fit = stats.logser(*result.params).pmf(IA.bins + 1)

            # Fit decay with MSE
            if fit_id == 3:
                dist_name = "Exponential with MSE"

                def func(x, A, K, C):
                    # C = 0
                    # return K * x + np.log(A)
                    return A * np.exp(-x / K) + C

                def objective(params, x, y):
                    return np.sum((func(x, *params) - y) ** 2)

                # Initial guess for parameters
                initial_guess = [1, 1, 0]
                # Enforce constraint: function can't give negative value
                cons = {
                    "type": "ineq",
                    "fun": lambda params: func(x, *params),
                }
                # cons = {}
                # Minimize the objective function with the constraint
                result = minimize(
                    objective, initial_guess, args=(x, p), constraints=cons
                )
                A, K, C = result.x
                print(f"A: {A:.5f}, K: {K:.5f}, C: {C:.5f}")
                params_str = f"A={A:.2g}<br>K={K:.2g}<br>C={C:.2g}"
                # params_str = f"{A:.2g},{K:.2g},{C:.2g}"
                y_fit = func(IA.midbins, A, K, C)

            if fit_id == 4:
                # Fit Boltzmann with MSE
                dist_name = "Boltzmann with MSE"

                def func(x, lambda_):
                    # C = 0
                    # return K * x + np.log(A)
                    return (
                        (1 - np.exp(-lambda_))
                        * np.exp(-lambda_ * x)
                        / (1 - np.exp(-lambda_ * IA.card))
                    )

                def objective(params, x, y):
                    return np.sum((func(x, *params) - y) ** 2)

                # Initial guess for parameters
                initial_guess = [0.8]
                # Enforce constraint: function can't give negative value
                cons = {
                    "type": "ineq",
                    "fun": lambda params: func(x, *params),
                }
                # cons = {}
                # Minimize the objective function with the constraint
                result = minimize(
                    objective, initial_guess, args=(x, p), constraints=cons
                )
                [lambda_] = result.x
                print(f"lambda_={lambda_:.5g}")
                params_str = f"param={lambda_:.4g}"
                y_fit = func(IA.midbins, result.x)
            return y_fit, dist_name, params_str

        y_fit_plot, dist_name, params_str = get_y_for_fit(fit_id)
        cpt_ia_ar[:, idx, fit_id - 1] = y_fit_plot

        fig.add_trace(
            go.Bar(
                x=x,
                y=p,
                name="Inferred inactive alveoli data",
                marker=dict(color="#0072b2"),
                legendgroup="2",
                showlegend=True if idx == 0 else False,
            ),
            col=1,
            row=idx + 1,
        )
        fig.add_trace(
            go.Bar(
                x=IA.midbins,
                y=y_fit_plot,
                name=f"Fitted {dist_name} distribution",
                marker=dict(color="#d55e00"),
                legendgroup="1",
                showlegend=True if idx == 0 else False,
            ),
            col=1,
            row=idx + 1,
        )
        fig.add_trace(
            go.Bar(
                x=x,
                y=p,
                name="Inactive alveoli data",
                marker=dict(color="#0072b2"),
                legendgroup="2",
                showlegend=False,
            ),
            col=2,
            row=idx + 1,
        )
        fig.add_trace(
            go.Bar(
                x=IA.midbins,
                y=y_fit_plot,
                name=f"Fitted {dist_name} distribution",
                marker=dict(color="#d55e00"),
                legendgroup="1",
                showlegend=False,
            ),
            col=2,
            row=idx + 1,
        )

        fig.update_yaxes(
            title=f"Probability<br><br>AR={ar_group:g}%<br>n={df_tmp['IA sample'].shape[0]}<br>{params_str}",
            col=1,
            row=idx + 1,
            range=[0, 0.64],
        )
        fig.update_yaxes(
            type="log", col=2, row=idx + 1, range=[np.log(10e-5), np.log(1)]
        )

    fig.update_xaxes(tickvals=IA.midbins)
    title = f"AR-IA CPT with {dist_name} fit - distribution of inactive alveoli (IA) given each bin of airway resistance ({len(ar_groups)} AR groups)"

    fig.update_layout(
        width=1600,
        height=3500,
        font=dict(size=9),
        # title=f"P({IA.name} | Airway resistance in % (ar_col))",
        title=f"{title}<br> using {n_samples} pairs of samples drawn from AR and IA's inferred distributions as input data",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )
    fig.update_annotations(font_size=10)
    # fig.show()
    fig.write_image(
        dh.get_path_to_main()
        # + f"PlotsBreathe/AR_modelling/CPT IA-AR - 22 groups/{title}.pdf"
        + f"PlotsBreathe/AR_modelling/CPT IA-AR - 22 groups - with Dirichlet fake data/{title}.pdf"
    )

# An Botlzmann fit is not ideal: with an exponential decay, the decay rate has to be small enough to prevent 0 probabilities for high IA values
# Exponential decay is too strong
# Try log decay

In [ ]:
cpt_ia_ar.shape

In [ ]:
# Test
fig = go.Figure()
lambda_, N = 0.3, 100
rv1 = stats.boltzmann(lambda_, N, loc=0)
rv2 = stats.logser(p=0.7)
rv3 = stats.poisson(mu=10)
res1 = rv1.pmf(IA.bins)
res2 = rv2.pmf(IA.bins + 1)
# res3 = rv3.pmf(IA.bins+1)
fig.add_trace(go.Bar(x=IA.midbins, y=res1 / sum(res1), name="Boltzmann"))
fig.add_trace(go.Bar(x=IA.midbins, y=res2 / sum(res2), name="Log-series"))
# fig.add_trace(go.Bar(x=IA.midbins, y=res3/sum(res3), name='Poisson'))
# rv2 = stats.planck()
fig.show()

## Analyse IA-AR cpt (with AR rebinned)

In [ ]:
import src.inference.helpers as ih

AR_rebinned = mh.VariableNode(
    "Airway resistance (%)", 0, 90, 4, prior={"type": "uniform"}
)

# cpt_ia_ar = cpt.get_cpt([IA, AR])
print(cpt_ia_ar.shape)
print("IA dim", IA.card)

In [ ]:
# Plot cpt using import src.models.cpts.helpers as cpt_helpers.plot_2d_cpt

# Create subplots
# n_ar_bins = AR.card
# n_ar_bins = len(ar_groups)

# fig = make_subplots(rows=n_ar_bins, cols=1, shared_xaxes=True, vertical_spacing=0.005)

# for i in range(n_ar_bins):
#     p = cpt_ia_ar[:, i, 1]
#     ih.plot_histogram(fig, IA, p, IA.a, IA.b, i+1, 1, colour='#0072b2', annot=False)
#     fig.update_yaxes(title_text=f"AR={AR_rebinned.midbins[i]:2g}%", row=i + 1, col=1)

# title = f"IA-AR CPT with Dirichlet prior - IA given AR ({len(ar_groups)} AR groups)"

# fig.update_layout(
#     title=title,
#     width=500,
#     height=1300,
#     showlegend=False,
#     font=dict(size=8),
# )
# # fig.show()
# fig.write_image(
#     dh.get_path_to_main()
#     + f"PlotsBreathe/AR_modelling/{title}.pdf"
# )

In [ ]:
# Plot cpt using import src.models.cpts.helpers as cpt_helpers.plot_2d_cpt

# Create subplots
fig = make_subplots(rows=IA.card, cols=1, shared_xaxes=True, vertical_spacing=0.003)

for i in range(IA.card):
    p = cpt_ia_ar[i, :, 1] / sum(cpt_ia_ar[i, :, 1])
    ih.plot_histogram(
        fig,
        AR_rebinned,
        p,
        AR_rebinned.a,
        AR_rebinned.b,
        i + 1,
        1,
        colour="#0072b2",
        annot=False,
    )
    fig.update_yaxes(title_text=f"IA={IA.midbins[i]}%", row=i + 1, col=1)
    # Add tickvals on x axis
    fig.update_xaxes(tickvals=ar_groups, row=i + 1, col=1)
    fig.add_shape(
        type="rect",
        x0=76,
        x1=90,
        y0=0,
        y1=0.37,
        line=dict(color="red", width=1),
        fillcolor="red",
        opacity=0.2,
        row=i + 1,
        col=1,
    )

fig.update_xaxes(title_text="Airway resistance midbin (%)", row=IA.card, col=1)
fig.update_yaxes(range=[0, 0.37], tickvals=[0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35])

title = f"IA-AR CPT with Dirichlet prior - AR given IA ({len(ar_groups)} AR groups)"

fig.update_layout(
    title=title,
    width=600,
    height=2500,
    showlegend=False,
    font=dict(size=8),
)
# fig.show()
fig.write_image(dh.get_path_to_main() + f"PlotsBreathe/AR_modelling/{title}.pdf")

## Format CPT to full AR resolution, analyse it and save

In [ ]:
import src.models.cpts.helpers as cpt

cpt_ia_ar.shape

print(AR.card)
print(AR_rebinned.card)

In [ ]:
# Reformat the AR dim for 45 bins instead of 22
cpt_ia_ar_final = np.zeros([IA.card, AR.card])

# Each bin is replicated 2 times
for i in range(cpt_ia_ar.shape[1] - 1):
    for j in [0, 1]:
        print(i * 2 + j)
        cpt_ia_ar_final[:, i * 2 + j] = cpt_ia_ar[:, i, 1]

# After AR 72-76%, replicate the same data
print(f"Unreliable data at bin: {AR.get_bin_for_value(76)}")
for i in np.arange(38, AR.card):
    cpt_ia_ar_final[:, i] = cpt_ia_ar_final[:, 37]

In [ ]:
# Create subplots
plot_bins = range(AR.card)
vspace = 0.003
height = 2500
title = (
    f"IA-AR full CPT with Dirichlet prior - IA given AR ({len(ar_groups)} AR groups)"
)

# plot_bins = np.arange(0, AR.card, 6)
# vspace=0.02
# height=700
# title = f"IA-AR full CPT with Dirichlet prior - IA given AR ({len(ar_groups)} AR groups) - summary"

fig = make_subplots(
    rows=len(plot_bins), cols=1, shared_xaxes=True, vertical_spacing=vspace
)

for i, idx in enumerate(plot_bins):
    p = cpt_ia_ar_final[:, idx]
    ih.plot_histogram(fig, IA, p, IA.a, IA.b, i + 1, 1, colour="#0072b2", annot=False)
    fig.update_yaxes(
        title_text=f"{AR.midbins[idx]:2g}%", row=i + 1, col=1, range=[0, 0.6]
    )

fig.update_layout(
    title=title,
    width=500,
    height=height,
    showlegend=False,
    font=dict(size=8),
)
# # fig.show()
# fig.write_image(
#     dh.get_path_to_main()
#     + f"PlotsBreathe/AR_modelling/{title}.pdf"
# )

In [ ]:
# Create subplots
plot_bins = IA.bins.astype(int)
vspace = 0.003
height = 2500
title = (
    f"IA-AR full CPT with Dirichlet prior - AR given IA ({len(ar_groups)} AR groups)"
)

# plot_bins = np.arange(0, IA.card, 4)
# vspace=0.02
# height=700
# title = f"IA-AR full CPT with Dirichlet prior - AR given IA ({len(ar_groups)} AR groups) - summary"


fig = make_subplots(
    rows=len(plot_bins), cols=1, shared_xaxes=True, vertical_spacing=vspace
)

for i, idx in enumerate(plot_bins):
    p = cpt_ia_ar_final[idx, :] / sum(cpt_ia_ar_final[idx, :])
    ih.plot_histogram(fig, AR, p, AR.a, AR.b, i + 1, 1, colour="#0072b2", annot=False)
    fig.update_yaxes(title_text=f"IA={IA.midbins[idx]}%", row=i + 1, col=1)
    # Add tickvals on x axis
    fig.update_xaxes(tickvals=ar_groups, row=i + 1, col=1)
    fig.add_shape(
        type="rect",
        x0=76,
        x1=90,
        y0=0,
        y1=0.37,
        line=dict(color="yellow", width=1),
        fillcolor="yellow",
        opacity=0.2,
        row=i + 1,
        col=1,
    )

fig.update_xaxes(title_text="Airway resistance midbin (%)", row=IA.card, col=1)
fig.update_yaxes(range=[0, 0.1], tickvals=[0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35])

fig.update_layout(
    title=title,
    width=600,
    height=height,
    showlegend=False,
    font=dict(size=8),
)
# fig.show()
fig.write_image(dh.get_path_to_main() + f"PlotsBreathe/AR_modelling/{title}.pdf")

In [ ]:
# Let's try this cpt
import src.models.cpts.helpers as cpt

cpt.save_cpt([IA, AR], cpt_ia_ar_final, "_Dirichlet_prior")